In [1]:
import os
import json
from collections import OrderedDict

from evaluate_lanenet_on_tusimple import eval_lanenet
from modules.homo_util import homoify
from modules.video_util import videoify
%matplotlib inline

In [2]:
DATA_PATH = "../test_set"
JSON_PATH = "outputs/json/"
SAVE_DIR = "outputs/seg_results/"
VID_DIR = "outputs/vids/"

CONST_IMAGE = (1312, 1312)
LANENET_WEIGHTS="weights/tusimple_lanenet.ckpt"

In [3]:
# frames_dir = "../test_set/clips/0531/1492626499813320696"
# frames_dir = "../test_set/clips/0530/1492626191132352208_0"
frames_dir = "../test_set/clips/0601/1494452387590928848"

In [4]:
clip_num = "_".join(frames_dir.split("/")[3:])
save_json = os.path.join(JSON_PATH, clip_num + ".json")
vid_path = os.path.join(VID_DIR, clip_num + ".mp4")
clip_num

'0601_1494452387590928848'

Detecting Lanes using lanenet

In [5]:
print("Detecting lanes")
lanes_json = eval_lanenet(frames_dir, LANENET_WEIGHTS, SAVE_DIR, save_json)
print("Lanes detected")
lanes_json = OrderedDict(sorted(lanes_json.items(), key=lambda x: int(x[0].split("/")[-1].split(".")[0])))

Detecting lanes


100%|██████████| 20/20 [00:16<00:00,  1.19it/s]

Lanes detected


In [6]:
lanes_json.keys()

odict_keys(['clips/0601/1494452387590928848/1.jpg', 'clips/0601/1494452387590928848/2.jpg', 'clips/0601/1494452387590928848/3.jpg', 'clips/0601/1494452387590928848/4.jpg', 'clips/0601/1494452387590928848/5.jpg', 'clips/0601/1494452387590928848/6.jpg', 'clips/0601/1494452387590928848/7.jpg', 'clips/0601/1494452387590928848/8.jpg', 'clips/0601/1494452387590928848/9.jpg', 'clips/0601/1494452387590928848/10.jpg', 'clips/0601/1494452387590928848/11.jpg', 'clips/0601/1494452387590928848/12.jpg', 'clips/0601/1494452387590928848/13.jpg', 'clips/0601/1494452387590928848/14.jpg', 'clips/0601/1494452387590928848/15.jpg', 'clips/0601/1494452387590928848/16.jpg', 'clips/0601/1494452387590928848/17.jpg', 'clips/0601/1494452387590928848/18.jpg', 'clips/0601/1494452387590928848/19.jpg', 'clips/0601/1494452387590928848/20.jpg'])

Yolo

In [7]:
YOLO_WEIGHTS = "../yolov7/yolov7.pt"
YOLO_PATH = "outputs/yolo"
CONF = 0.25
SOURCE = frames_dir
IMG_SIZE = 1280

In [8]:
os.system(f"python ../yolov7/detect.py --weights {YOLO_WEIGHTS} --source {SOURCE} --conf {CONF} --img-size {IMG_SIZE} --save-txt --project {YOLO_PATH}")
print("YOLO done")

YOLO done


Homography

In [9]:
final_frames = homoify(lanes_json, DATA_PATH, YOLO_PATH, CONST_IMAGE)
print("Final Frames ready for video writing")

Final Frames ready for video writing


Writing Video

In [10]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [11]:
def read_imgs(frames_dir):
    frames = [frames_dir + "/" + x for x in os.listdir(frames_dir)]
    frames = sorted(frames, key=lambda x: int(x.split("/")[-1].split(".")[0]))
    frames = [cv2.imread(x) for x in frames]
    frames = [cv2.cvtColor(x, cv2.COLOR_BGR2RGB) for x in frames]
    return frames

def TextingOnImg(const_image_size, num_lanes, num_objs):
    black_img = np.zeros(const_image_size, np.uint8)
    text1 = "Textual summary of current scene: "
    text2 = f"Total number of lanes: {num_lanes}"
    text3 = f"Total number of objects: {num_objs}"

    black_img = cv2.putText(black_img, text1, (20, 100), cv2.FONT_HERSHEY_PLAIN, 3, (255, 255, 255), 3)
    black_img = cv2.putText(black_img, text2, (20, 100 + 50), cv2.FONT_HERSHEY_PLAIN, 3, (255, 255, 255), 3)
    black_img = cv2.putText(black_img, text3, (20, 100 + 100), cv2.FONT_HERSHEY_PLAIN, 3, (255, 255, 255), 3)

    return black_img

def cat_images(top_view, blackImg, instanceSeg, source, max_w, max_h):
    canvas = np.zeros((max_h, max_w, 3), np.uint8)
    canvas[:source.shape[0], :source.shape[1]] = source
    canvas[:instanceSeg.shape[0], source.shape[1]:source.shape[1] + instanceSeg.shape[1]] = instanceSeg
    canvas[source.shape[0]:source.shape[0] + top_view.shape[0], :top_view.shape[1]] = top_view
    canvas[source.shape[0]:source.shape[0] + blackImg.shape[0], top_view.shape[1]:top_view.shape[1] + blackImg.shape[1]] = blackImg

    return canvas

def videoify(clip_num, final_frames, vid_path, save_dir, const_image=(1312, 1312)):
    laneNetResultsPath = os.path.join(save_dir,clip_num)
    instanceSegResultsPath = f"{laneNetResultsPath}/instance_seg"
    sourceResultsPath = f"{laneNetResultsPath}/result"
    instanceSeg = read_imgs(instanceSegResultsPath)
    source = read_imgs(sourceResultsPath)
    instanceSeg = [cv2.resize(x, (1280, 720)) for x in instanceSeg]

    max_w = max(CONST_IMAGE[1], instanceSeg[0].shape[1], source[0].shape[1]) * 2
    max_h = CONST_IMAGE[0] + source[0].shape[0]

    vid = cv2.VideoWriter(vid_path, cv2.VideoWriter_fourcc(*'mp4v'), 4, (max_w, max_h))
    for idx in range(len(final_frames)):
        top_view = final_frames[idx]['img']
        num_lanes = final_frames[idx]['num_lanes']
        num_objs = final_frames[idx]['num_objs']
        blackImg = TextingOnImg((const_image[0], const_image[1], 3), num_lanes, num_objs)
        canvas = cat_images(top_view, blackImg, instanceSeg[idx], source[idx], max_w, max_h)

        vid.write(canvas)
    vid.release()
    
    return "Video Written"

In [12]:
videoify("/".join(frames_dir.split("/")[-2:]), final_frames, vid_path, SAVE_DIR, CONST_IMAGE)
print(f"Video written in {vid_path}")

Video written in outputs/vids/0601_1494452387590928848.mp4
